In [34]:
1+1

2

In [35]:
import os
%pwd

'/Users/brunoferreira/Downloads'

In [36]:
os.chdir("../")
%pwd

'/Users/brunoferreira'

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [38]:
%pwd

'/Users/brunoferreira'

In [39]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
            
        )

        return model_evaluation_config

In [41]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk

import torch
import pandas as pd
from tqdm import tqdm

In [42]:
import evaluate



class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = evaluate.load('rouge')

        #rouge_metric = rouge_metric

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        # Directly use the scores without accessing fmeasure or mid
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

In [44]:
from pathlib import Path
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate

def _me_generate_batch_sized_chunks(self, list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i + batch_size]

def _me_calculate_metric_on_test_ds(
    self, dataset, metric, model, tokenizer,
    batch_size=2, device="cpu", column_text="dialogue", column_summary="summary"
):
    article_batches = list(_me_generate_batch_sized_chunks(self, dataset[column_text], batch_size))
    target_batches = list(_me_generate_batch_sized_chunks(self, dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(
            article_batch,
            max_length=1024,
            truncation=True,
            padding=True,
            return_tensors="pt"
        )

        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8,
            num_beams=8,
            max_length=128
        )

        decoded_summaries = [
            tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for s in summaries
        ]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    return metric.compute()

def _me_load_model_and_tokenizer(self, device):
    project_root = Path("/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer")

    model_path = Path(self.config.model_path)
    tokenizer_path = Path(self.config.tokenizer_path)

    if not model_path.is_absolute():
        model_path = project_root / model_path
    if not tokenizer_path.is_absolute():
        tokenizer_path = project_root / tokenizer_path

    model_path = model_path.resolve()
    tokenizer_path = tokenizer_path.resolve()

    print("Resolved model_path:", model_path)
    print("Resolved tokenizer_path:", tokenizer_path)

    if model_path.exists() and tokenizer_path.exists():
        try:
            tokenizer = AutoTokenizer.from_pretrained(str(tokenizer_path), local_files_only=True)
            model = AutoModelForSeq2SeqLM.from_pretrained(str(model_path), local_files_only=True).to(device)
            print("Modelo local carregado com sucesso.")
            return model, tokenizer, "local"
        except Exception as e:
            print(f"Falha no modelo local ({type(e).__name__}): {e}")

    print("Fallback para modelo base: google/pegasus-cnn_dailymail")
    base_model = "google/pegasus-cnn_dailymail"
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    model = AutoModelForSeq2SeqLM.from_pretrained(base_model).to(device)
    return model, tokenizer, "hf_base"

def _me_evaluate(self):
    device = "cpu"
    project_root = Path("/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer")

    data_path = Path(self.config.data_path)
    metric_file_name = Path(self.config.metric_file_name)

    if not data_path.is_absolute():
        data_path = (project_root / data_path).resolve()
    if not metric_file_name.is_absolute():
        metric_file_name = (project_root / metric_file_name).resolve()

    print("Resolved data_path:", data_path, data_path.exists())

    dataset_samsum_pt = load_from_disk(str(data_path))
    rouge_metric = evaluate.load("rouge")

    model_pegasus, tokenizer, model_source = _me_load_model_and_tokenizer(self, device=device)

    test_sample = dataset_samsum_pt["test"].select(range(10))
    score = _me_calculate_metric_on_test_ds(
        self,
        dataset=test_sample,
        metric=rouge_metric,
        model=model_pegasus,
        tokenizer=tokenizer,
        batch_size=2,
        device=device,
        column_text="dialogue",
        column_summary="summary",
    )

    rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    rouge_dict = {rn: score[rn] for rn in rouge_names}

    metric_file_name.parent.mkdir(parents=True, exist_ok=True)
    pd.DataFrame(rouge_dict, index=["pegasus"]).to_csv(metric_file_name, index=False)

    print("Model source:", model_source)
    print("ROUGE:", rouge_dict)
    print("Métricas salvas em:", metric_file_name)

# HOTFIX da classe em memória
ModelEvaluation.generate_batch_sized_chunks = _me_generate_batch_sized_chunks
ModelEvaluation.calculate_metric_on_test_ds = _me_calculate_metric_on_test_ds
ModelEvaluation._load_model_and_tokenizer = _me_load_model_and_tokenizer
ModelEvaluation.evaluate = _me_evaluate

print("ModelEvaluation hotfix aplicado (end-to-end).")


ModelEvaluation hotfix aplicado (end-to-end).


In [45]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
model_evaluation_config.evaluate()

[2026-02-22 21:10:21,418: INFO: common: yaml file: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/config/config.yaml loaded successfully]
[2026-02-22 21:10:21,422: INFO: common: yaml file: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/params.yaml loaded successfully]
[2026-02-22 21:10:21,422: INFO: common: created directory at: artifacts]
[2026-02-22 21:10:21,423: INFO: common: created directory at: artifacts/model_evaluation]
Resolved data_path: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/data_transformation/samsum_dataset True


Resolved model_path: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/model_trainer/pegasus-samsum-model
Resolved tokenizer_path: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/model_trainer/tokenizer
Falha no modelo local (SafetensorError): Error while deserializing header: incomplete metadata, file not fully covered
Fallback para modelo base: google/pegasus-cnn_dailymail
[2026-02-22 21:10:23,787: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-22 21:10:23,805: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-22 21:10:23,930: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-22

Loading weights: 100%|██████████| 680/680 [00:00<00:00, 1305.33it/s, Materializing param=model.shared.weight]                                   
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
PegasusForConditionalGeneration LOAD REPORT from: google/pegasus-cnn_dailymail
Key                                  | Status  | 
-------------------------------------+---------+-
model.decoder.embed_positions.weight | MISSING | 
model.encoder.embed_positions.weight | MISSING | 

Notes:
- MISSING	:t

[2026-02-22 21:10:29,952: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/generation_config.json "HTTP/1.1 307 Temporary Redirect"]


[2026-02-22 21:10:29,954: WARNING: _http: Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.]
[2026-02-22 21:10:29,972: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/generation_config.json "HTTP/1.1 200 OK"]


100%|██████████| 5/5 [02:36<00:00, 31.21s/it]

[2026-02-22 21:13:06,104: INFO: rouge_scorer: Using default tokenizer.]


Model source: hf_base
ROUGE: {'rouge1': np.float64(0.012817150063051702), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.012595400039055649), 'rougeLsum': np.float64(0.012462246758877714)}
Métricas salvas em: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/model_evaluation/metrics.csv
